# Shakes and Quakes
### *Ordinal Modelling of Earthquake-Induced Building Damages*
### Patricio Hernandez Senosiain
---

## Dependencies

In [ ]:
# Scikit learn - preprocessing
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, RobustScaler, FunctionTransformer, LabelEncoder

# Scikit learn - model selection
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, KFold, cross_val_score, cross_validate

# Scikit learn - model evaluation
from sklearn.metrics import f1_score, make_scorer, confusion_matrix, ConfusionMatrixDisplay, classification_report

# Scikit learn - modelling
from sklearn.compose import ColumnTransformer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.multiclass import OneVsOneClassifier, OutputCodeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.tree import ExtraTreeClassifier, DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.decomposition import PCA


# CatBoost - modelling
from catboost import CatBoostClassifier

# XGBoost - modelling
from xgboost import XGBClassifier

# LightGBM - modelling
from lightgbm import LGBMClassifier

# Category Encoders - feature encoding
from category_encoders import TargetEncoder, LeaveOneOutEncoder, CatBoostEncoder
from category_encoders.wrapper import PolynomialWrapper

# Pandas and Numpy - data handling
import pandas as pd
import numpy as np

# Plotly, Matplotlib, and Seaborn - visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from qbstyles import mpl_style

from sklearn import set_config

In [ ]:
# Matplotlib theme
#mpl_style(dark=True)

# Declaring standard Seaborn color palette
standard_palette = []
for i in range(10):
    standard_palette += list(sns.color_palette('muted'))


# Convergence warning disabling
from sklearn.exceptions import ConvergenceWarning
from warnings import simplefilter
    #simplefilter("ignore", category=ConvergenceWarning)

# Setting a random seed
SEED = 105

## Data Retrieval

In [ ]:
train_features = pd.read_csv('data/train_values.csv')
test_features = pd.read_csv('data/test_values.csv') 
train_target = pd.read_csv('data/train_labels.csv') 

## General Overview

In [ ]:
print('Train Features data:')
train_features.info(verbose=False)
print('')
print('Test Features data:')
test_features.info(verbose=False)
print('Train Target data:')
train_target.info(verbose=False) 

In [ ]:
# Joining train target to features data
data = train_features.merge(train_target, on='building_id' )
data = data.drop(columns='building_id')

# Separating 'Id' column for test observations
ID = test_features['building_id']

# Joining test and training datasets
data = data.append(test_features, sort=False)
data = data.drop(columns='building_id') 

# Printing overview
data.info()

In [ ]:
# Classifying features
loc_feats = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id' ]
cont_feats = ['area_percentage', 'height_percentage']
count_feats = ['count_floors_pre_eq', 'age', 'count_families']
cat_feats = ['land_surface_condition', 'foundation_type', 'roof_type', 
             'ground_floor_type', 'other_floor_type', 'position', 
             'plan_configuration', 'legal_ownership_status']

binary_feats = data.drop(columns=cont_feats+count_feats+cat_feats+loc_feats).columns.tolist()
binary_feats.remove('damage_grade') 

## Feature Engineering

In [ ]:
data['area_percentage'] = np.log1p(data['area_percentage']) 

data['age'] = np.log1p(data['age']/5)


level_1 = range(31)
level_2 = range(1428)
level_3 = range(12568)

loc_cats = [level_1, level_2, level_3]


## Evaluation Metrics


Although this is an ordinal regression problem, the metric used by Driven Data to evaluate models for this competition is the micro averaged F1 score across the three classes:

$$ F_{micro} \; = \; \frac{2 \cdot P_{micro} \cdot R_{micro} }{P_{micro} + R_{micro} }$$

where $P_{micro}$ and $R_{micro}$ stand for the precision and recall metrics :


$$ P_{micro} \;=\; \frac{\sum^3_{k=1} TP_k}{\sum^3_{k=1}TP_k + FP_k} \;,\; R_{micro} \;=\; \frac{\sum^3_{k=1}TP_k}{\sum^3_{k=1}TP_k + TN_k} $$

In [ ]:
def model_score(y_true, y_pred):
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    ConfusionMatrixDisplay(cm).plot()
    print('Confusion Matrix')
    print(cm)
    print('')
    
    # Classification report
    print('Classification Report')
    print(classification_report(y_true, y_pred))
    print('')
    print('Micro F1 Score')
    print(f1_score(y_true, y_pred, average='micro'))
    
    return f1_score(y_true, y_pred, average='micro')
 

## Defining Feature Matrices and Target Vector

In [ ]:
# Separating training and test matrices
train = data[data['damage_grade'].isnull()==False]
test = data[data['damage_grade'].isnull()]

# Creating feature matrix and target vector from training sample
X_train = train.drop(columns='damage_grade')
y_train = train['damage_grade']

# Creating feature matrix from test sample
X_test = test.drop(columns='damage_grade')


## Hierarchical Location Encoding

In [ ]:
X_loc = X_train[loc_feats].copy() 
X_loc_test = X_test[loc_feats].copy() 

y_loc = pd.get_dummies(y_train, prefix='y', drop_first=True) 

X_loc = X_loc.join(y_loc)
X_loc['y'] = y_train

w=100
for c in [2.0, 3.0]:
    for i in range(1,4):
        level = f'geo_level_{i}_id'
        prior = f'level_{i-1}_y_{round(c)}'
        posterior =  f'level_{i}_y_{round(c)}'
        target = f'y_{c}'

        if i==1:
            prior_mean = X_loc[target].mean()
        else:
            prior_mean = X_loc.groupby(level)[prior].mean() 

        # Compute the number of values and the mean of each group
        agg = X_loc.groupby(level)[target].agg(['count', 'mean'])
        count = agg['count']
        mean = agg['mean']

        # Compute the Laplace smoothened means
        smooth = (count * mean + w * prior_mean) / (count + w)

        # Replace each value by the according smoothed means
        X_loc[posterior] = X_loc[level].map(smooth)
        X_loc_test[posterior] = X_loc_test[level].map(smooth)

        # Replacing NA's in test data with prior
        if i>1:
            X_loc_test[posterior].fillna(X_loc_test[prior], inplace=True)

X_loc[['y', 'level_3_y_2', 'level_3_y_3']].head()

In [ ]:
post_feats = ['post_2', 'post_3']

X_train['post_2'] = X_loc['level_3_y_2']
X_train['post_3'] = X_loc['level_3_y_3']

X_test['post_2'] = X_loc_test['level_3_y_2']
X_test['post_3'] = X_loc_test['level_3_y_3']

X_train.head()

## General Preprocessing Pipeline 

In [ ]:
# General preprocessing pipeline
gen_pp = ColumnTransformer(
    [('log-transform', FunctionTransformer(np.log1p), ['age', 'area_percentage']), 
     ('categorical', OneHotEncoder(), cat_feats) 
    ], 
    remainder='passthrough'
)

## Logistic Regression

In [ ]:
# Creating logistic regression preprocessing pipeline
logreg_pp = ColumnTransformer(
    [('continuous', make_pipeline(RobustScaler(), PCA(whiten=True)), cont_feats),
     ('categorical', OneHotEncoder(), cat_feats),   
     ('location', OneHotEncoder(categories=loc_cats), loc_feats),
     ('posterior', 'drop', post_feats)
    ], 
    remainder='passthrough'
)

# Creating Logistic Regression pipeline --- hyperparameter has been optimized
logreg_pipe = Pipeline([('specific', logreg_pp),
                        ('model', LogisticRegression(multi_class='multinomial',
                                                     max_iter=1600,
                                                     random_state=SEED))])

set_config(display='diagram')
logreg_pipe  

## SVM 

In [ ]:
# Creating SVM preprocessing pipeline
svm_pp = ColumnTransformer(
    [('continuous', make_pipeline(RobustScaler(), PCA(whiten=True)), cont_feats),  
     ('categorical', OneHotEncoder(), cat_feats), 
     ('location', OneHotEncoder(categories=loc_cats), loc_feats),
     ('posterior', 'drop', post_feats)
    ], 
    remainder='passthrough'
)

svm_model = OneVsOneClassifier(LinearSVC(random_state=SEED, C=6.5, dual=False))

svm_params = {'model__estimator__C': np.logspace(-1,1,20) }

# Creating SVM pipeline --- hyperparameter has been optimized
svm_pipe = Pipeline([('processing', svm_pp),
                     ('model', svm_model)])

set_config(display='diagram')
svm_pipe  

## Random Forest 

In [ ]:
# Creating Random Forest preprocessing pipeline
rf_pp = ColumnTransformer( 
    [('continuous', make_pipeline(RobustScaler(), PCA(whiten=True)), cont_feats),  
     ('categorical', OneHotEncoder(), cat_feats), 
     ('location', 'drop', loc_feats),
     ('posterior', 'passthrough', post_feats) 
    ], 
    remainder='passthrough') 

# Hyperparameters for Random Forest regressor
rf_params = {'model__n_estimators': np.arange(200, 350, 10),
             'model__max_depth': np.arange(20, 60, 2),
             'model__min_samples_split': np.arange(36, 160, 2),
             'model__min_samples_leaf': np.arange(1, 14, 1) 
             }

# Creating Random Forest pipeline  
rf_pipe = Pipeline([('specific', rf_pp), 
                    ('model', RandomForestClassifier(n_estimators=350,
                                                     max_depth=40, min_samples_split=46,
                                                     min_samples_leaf=8, random_state=SEED))])
                    
set_config(display='diagram')
rf_pipe              

## Multi-Layer Perceptron

In [ ]:
# Creating MLP preprocessing pipeline
mlp_pp = ColumnTransformer( 
    [('continuous', make_pipeline(RobustScaler(), PCA(whiten=True)), cont_feats),  
     ('categorical', OneHotEncoder(drop='first'), cat_feats), 
     ('location', 'drop', loc_feats),
     ('posterior', 'passthrough', post_feats)
    ], 
    remainder='passthrough') 

# Creating MLP pipeline
mlp_pipe = Pipeline([('preprocessing', mlp_pp),
                     ('model', MLPClassifier())])

set_config(display='diagram')
mlp_pipe

## CatBoost 

In [ ]:
# Creating LightGBM preprocessing pipeline
catboost_pp = ColumnTransformer(
    [('continuous', make_pipeline(RobustScaler(), PCA(whiten=True)), cont_feats),  
     ('categorical', 'passthrough', cat_feats), 
     ('location', 'passthrough', loc_feats),
     ('posterior', 'drop', post_feats) 
    ], 
    remainder='passthrough') 

# Creating CatBoost pipeline
catboost_pipe = Pipeline([('model', CatBoostClassifier(cat_features=cat_feats + loc_feats, 
                                                       loss_function='MultiClass'))])
set_config(display='diagram')
catboost_pipe

## XGBoost 

In [ ]:
# Creating XGBoost preprocessing pipeline
xgboost_pp = ColumnTransformer(
    [('continuous', make_pipeline(RobustScaler(), PCA(whiten=True)), cont_feats),  
     ('categorical', OneHotEncoder(), cat_feats), 
     ('location', 'drop', loc_feats),
     ('posterior', 'passthrough', post_feats)
    ], 
    remainder='passthrough') 
# Creating XGBoost pipeline
xgboost_pipe = Pipeline([('preprocessing', xgboost_pp), 
                         ('model', XGBClassifier(objective='multi:softmax', 
                                                     num_class=3))])

set_config(display='diagram')
xgboost_pipe

## LightGBM 

In [ ]:
# Creating LightGBM preprocessing pipeline
lgbm_pp = ColumnTransformer(
    [('continuous', make_pipeline(RobustScaler(), PCA(whiten=True)), cont_feats),  
     ('categorical', OneHotEncoder(), cat_feats), 
     ('location', 'drop', loc_feats),
     ('posterior', 'passthrough', post_feats)
    ], 
    remainder='passthrough') 
    
# Creating LightGBM pipeline
lgbm_pipe = Pipeline([('preprocessing', lgbm_pp),
                          ('model', LGBMClassifier(objective='multiclass', 
                                                   num_class=3))])

set_config(display='diagram')
lgbm_pipe

## Model Cross-Validation

In [ ]:
## Running initial 10 - fold cross validation 
# note: CV does not work with catboost pipe
print('Running Cross-Validation ...')
#%time cv = cross_val_score(catboost_pipe, X_train.drop(columns=post_feats), y_train, scoring=make_scorer(model_score), cv=5, n_jobs=-1)
    
print('Finished running')
#print('Score: ', cv)

## Estimator Stacking

In [ ]:
# Preparing best estimators to pass into stacked model
final_estimators = [('CatBoost', catboost_pipe),
                    ('LightGBM', lgbm_pipe),
                    ('XGBoost', xgboost_pipe),
                    ('MLP', mlp_pipe), 
                    ('Random Forest', rf_pipe), 
                    ('SVM', svm_pipe), 
                    ('Logistic Regression', logreg_pipe)]

methods = {'CatBoost':'predict',
           'LightGBM':'predict',
           'XGBoost':'predict',
           'Random Forest':'predict', 
           'SVM':'decision_function', 
           'Logistic Regression':'predict_proba'}
#stack = StackingClassifier(estimators=final_estimators, cv=5)

#print('Running stacked estimator model...')
#%time cv = cross_val_score(stack, X_train, y_train, scoring=make_scorer(model_score), cv=5, n_jobs=-1)
#print('Finished running')
#print('Score: ', cv)

## Final model

In [ ]:
# Create estimator with optimal hyperparameters
final_model = StackingClassifier(final_estimator = LogisticRegression(multi_class='multinomial',
                                                                      max_iter=1600,
                                                                      random_state=SEED), 
                                 estimators=final_estimators, 
                                 n_jobs=1)

set_config(display='diagram')
final_model

## Final Predictions and Submission

In [ ]:
# Fit model with complete training sample
final_model.fit(X_train, y_train)

# Creating submission file
preds = final_model.predict(X_test)
preds = preds.flatten()
output = pd.DataFrame( {'building_id': ID, 'damage_grade': preds})
output = output.astype(int)
output.to_csv('submission.csv', index=False)